## 7. 基于MindSpore实现二维线性回归

本实验将实现基于MindSpore的二维线性回归，包括生成数据集、模型构建、模型预测。

### 1、实验目的
- 掌握如何使用MindSpore实现二维线性回归模型。
- 了解如何使用MindSpore的Adam优化器和MSE损失函数。
- 了解基于MindSpore的模型训练和模型预测。

### 2、整体模型介绍
- 二维线性回归原理

在机器学习领域，线性回归模型记为：
$$y=w_0x_0+w_1x_1+\cdots+w_nx_n+b=[w_0 w_1 w_2 \cdots w_n][x_0 x_1 x_2 \cdots x_n]^T+b$$
可以统一形式为：
$$y=\sum_{i=0}^{n}w_ix_i+b=w^Tx+b$$


并且我们定义损失函数来度量模型一次预测的好坏，即预测值$\widehat y$和真实值y的误差，线性损失函数一般取$L=\frac{1}{2}(y-\widehat y)^2$，平方损失函数的几何意义是欧氏距离。

之后我们便可进行模型训练，采用梯度下降方法求模型参数w，使损失函数最小。

梯度下降法顺着当前点梯度反方向，按规定步长$\alpha$进行迭代搜索，对第i个模型参数进行如下更新：
$$w_{i+1}=w_i-\alpha \frac{\partial L(w)}{\partial (w_i)}$$
因为
$$\frac{\partial L(w)}{\partial (w_i)}=-\sum_{j=0}^{m}[y^{(j)}-\sum_{i=0}^{n}w_i x_i^{(j)}-b]*x_i^{(j)}$$
所以
$$w_{i+1}=w_i+\alpha [\sum_{j=0}^{m}(y^{(j)}-\sum_{i=0}^{n}w_i x_i^{(j)}-b)*x_i^{(j)}]$$
对每个模型参数迭代训练直到收敛即可。因此二维线性回归的自变量为两个，模型便为：
$$y=w_0x_0+w_1x_1+b=w^Tx+b$$

因此我们需要定义一个模型Net实现二维线性回归功能。<br>
可以调用MindSpore的nn.MSELoss计算预测值和目标值之间的均方误差。<br>
调用MindSpore的nn.Adam计算最优参数。

### 3 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=2.4；Python环境=3.11。


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU |Linux-x86_64| MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|


### 4、数据处理
#### 4.1 数据准备
为了简单起见，我们将根据带有噪声的线性模型构造一个人造数据集。任务是使用这个有限样本的数据集来恢复这个模型的参数。我们需要生成一个包含1000个样本的数据集，每个样本包含从标准正态分布中采样的2个特征。 合成的数据集是一个矩阵$X∈R^{(1000×2)}$。我们使用线性模型参数$w =[2,-3.4]^T$、b=3.2和噪声项$\epsilon$生成数据集及其标签：
$y=Xw+b+\epsilon=w_0x_0+w_1x_1+b+\epsilon$

$\epsilon$可以视为模型预测和标签之间的观测误差，我们假设ϵ服从均值为0、标准差为0.01的正态分布。


random库实现了各种分布的伪随机数生成器；dtype是MindSpore数据类型的对象；mindspore.ops提供对神经网络层的各种操作；pyplot是常用的绘图模块，能很方便让用户绘制 2D 图表；MindSpore中的Tensor是张量，可放在gpu上加速。

In [1]:
import random
import numpy as np
import mindspore
from mindspore import dtype as mstype
import mindspore.ops as ops
from matplotlib import pyplot as plt
import sys
sys.path.append('..')

In [2]:
def synthetic_data(w, b, num_examples):  
    print((num_examples, len(w)))
    # 生成X
    X = np.random.normal(0, 1, (num_examples, len(w))).astype(np.float32)
    # y = Xw + b
    y = np.matmul(X, w) + b 
    # y = Xw + b + 噪声
    y += np.random.normal(0, 0.01, len(y)).astype(np.float32)          
    return X, y.reshape((-1, 1))

mindspore.set_seed(1)
true_w = np.array([2, -3.4]).astype(np.float32)
true_b = np.float32(4.2)
# 生成数据
features, labels = synthetic_data(true_w, true_b, 1000)

(1000, 2)


#### 4.2 数据加载
数据features中的每一行都包含一个二维数据样本，真实值labels中的每一行都包含一维标签值（一个标量）

In [3]:
print('features:', features[0:4],'\nlabel:', labels[0:4])

features: [[ 1.6243454  -0.6117564 ]
 [-0.5281718  -1.0729686 ]
 [ 0.86540765 -2.3015387 ]
 [ 1.7448118  -0.7612069 ]] 
label: [[ 9.533558]
 [ 6.794138]
 [13.751566]
 [10.271619]]


In [4]:
def split_dataset(data, labels, train_ratio=0.9):
    """
    将数据集划分为训练集和测试集
    :param data: NumPy数组，包含所有数据
    :param labels: NumPy数组，包含所有标签
    :param train_ratio: 浮点数，训练集的比例
    :return: 训练集数据，测试集数据，训练集标签，测试集标签
    """
    assert len(data) == len(labels), "数据和标签长度不匹配"
    num_samples = len(data)
    indices = np.random.permutation(num_samples)
    num_train_samples = int(num_samples * train_ratio)
    train_indices = indices[:num_train_samples]
    test_indices = indices[num_train_samples:]
    
    X_train = data[train_indices]
    y_train = labels[train_indices]
    X_test = data[test_indices]
    y_test = labels[test_indices]
    
    return X_train, X_test, y_train, y_test

In [5]:
train_features, test_features, train_labels, test_labels = split_dataset(features, labels)

In [6]:
len(train_features), len(test_features), len(train_labels), len(test_labels)

(900, 100, 900, 100)

使用MindSpore的GeneratorDataset创建可迭代数据。<br>
dataset模块提供了加载和处理数据集的API；numpy提供了一系列类NumPy接口。

In [7]:
from mindspore import dataset as ds  

class train_DatasetGenerator:
    def __init__(self):
        self.data = train_features
        self.label = train_labels

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)
        
batch_size = 4
dataset_generator = train_DatasetGenerator()
train_dataset = ds.GeneratorDataset(dataset_generator, ["data", "label"], shuffle=True)
# 将数据集中连续10条数据合并为一个批处理数据
train_dataset = train_dataset.batch(batch_size)  
print(len(train_dataset))

225


In [8]:
class test_DatasetGenerator:
    def __init__(self):
        self.data = test_features
        self.label = test_labels

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)

dataset_generator = test_DatasetGenerator()
test_dataset = ds.GeneratorDataset(dataset_generator, ["data", "label"], shuffle=True) 
print(len(test_dataset))

100


### 5、模型构建

模型构建分为定义实现二维线性回归功能的Net、用于计算预测值与标签值之间的均方误差MSELoss、Adam优化器。
- 定义模型Net

Net输入为X样本，计算出预测值$\widehat y$并返回。<br>
mindspore.nn用于构建神经网络中的预定义构建块或计算单元；Parameter 是 Tensor 的子类，当它们被绑定为Cell的属性时，会自动添加到其参数列表中，并且可以通过Cell的某些方法获取；mindspore.common.initializer用于初始化神经元参数。

In [9]:
import mindspore.nn as nn
from mindspore import Parameter
from mindspore.common.initializer import initializer, Zero, Normal


def linreg(x, w, b):
    # y = Xw+b
    return ops.matmul(x, w) + b    


class Net(nn.Cell):
    def __init__(self):
        super().__init__()
        self.w = Parameter(initializer(Normal(0.01, 0), (2, 1), mstype.float32))
        self.b = Parameter(initializer(Zero(), 1, mstype.float32))
        
    def construct(self, x):
        # y_hat = Xw+b
        y_hat = linreg(x, self.w, self.b)  
        return y_hat
    
    
# Net用于实现二维线性回归
net = Net()

- 使用MSE损失函数和Adam优化器

In [10]:
# 训练的epoch为3
num_epochs = 3
# 学习率为0.03
lr = 0.03
# Adam优化器
optimizer = nn.Adam(net.trainable_params(), learning_rate=lr)          
# 计算预测值与标签值之间的均方误差
loss_fn = nn.MSELoss()   

### 6、模型训练
Model是模型训练与推理的高阶接口，调用Model.train方法，传入数据集即可完成模型训练，其中LossMonitor()会监控训练的损失，并将epoch、step、loss信息打印出来，若loss变为NAN或INF，则会终止训练。

In [11]:
# 1. Define forward function
def forward_fn(data, label):
    logits = net(data)
    loss = loss_fn(logits, label)
    return loss, logits

# 2. Get gradient function
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

# 3. Define function of one-step training
def train_step(data, label):
    (loss, _), grads = grad_fn(data, label)
    optimizer(grads)
    return loss

def train(model, dataset):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 10 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
    test_loss /= num_batches
    print(f"Test: Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(net, train_dataset)
    test(net, test_dataset, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 14.060421  [  0/225]
loss: 21.583872  [ 10/225]
loss: 17.625809  [ 20/225]
loss: 35.062145  [ 30/225]
loss: 5.402611  [ 40/225]
loss: 29.139320  [ 50/225]
loss: 18.576349  [ 60/225]
loss: 7.512076  [ 70/225]
loss: 7.636040  [ 80/225]
loss: 5.560393  [ 90/225]
loss: 2.784903  [100/225]
loss: 1.728753  [110/225]
loss: 10.996404  [120/225]
loss: 7.493189  [130/225]
loss: 2.314899  [140/225]
loss: 2.995142  [150/225]
loss: 0.536584  [160/225]
loss: 0.522829  [170/225]
loss: 1.050164  [180/225]
loss: 0.975079  [190/225]
loss: 0.891810  [200/225]
loss: 0.564698  [210/225]
loss: 0.064260  [220/225]
Test: Avg loss: 0.250020 

Epoch 2
-------------------------------
loss: 0.081173  [  0/225]
loss: 0.130326  [ 10/225]
loss: 0.102847  [ 20/225]
loss: 0.161158  [ 30/225]
loss: 0.034924  [ 40/225]
loss: 0.084393  [ 50/225]
loss: 0.036734  [ 60/225]
loss: 0.011196  [ 70/225]
loss: 0.009928  [ 80/225]
loss: 0.010508  [ 90/225]
loss: 0.003295  [100/225]
lo

### 7、模型预测
训练3个epoch后输出w和b的估计误差，比较真实参数和通过训练学到的参数来评估训练的成功程度。

In [13]:
# w的真实值和训练值之差
print(f'w的估计误差: {true_w - net.trainable_params()[0].reshape(true_w.shape)}')  
# b的真实值和训练值之差
print(f'b的估计误差: {true_b - net.trainable_params()[1]}')                        

w的估计误差: [Tensor(shape=[], dtype=Float32, value= -3.29018e-05)
 Tensor(shape=[], dtype=Float32, value= -0.000446081)]
b的估计误差: [Tensor(shape=[], dtype=Float32, value= 0.00028801)]
